TODO
* ~~Load products quality dataset~~
 * Drop some unused columns
 * Add the description of the quality dataset (e.g role of variables, ..)
* Generate descriptive statistics
 * Use matlibplot
* Load and merge atlas datasets
 * List of countries
 * List of products (SITC4)
 * Countries and products metadata
  * Color
  * Full / short names
* ~~Load `vistk.py`~~
* Generate visualizations we find on The Atlas
 * Country treemap
 * Geo-maps
 * Products treemap
 * Stacked graph
 * ~~Scatterplot~~
 * Product space
* Generate new visualizations
 * Caterplot
* Include the product quality in the above visualizations

In [ ]:
import sys
sys.path.append("./modules")
import vistk
import pandas as pd
import json

In [ ]:
df = pd.read_csv('/Users/rvuillemot/Downloads/master_data.csv').sort(columns='year')

In [ ]:
df.head()

In [ ]:
dotplot = vistk.Dotplot(id='sitc4u', name='sitc4u', x='quality_exp', year=1984, color='color', 
                        group='community_id')
dotplot.draw(df[df['year'] == 1984])

In [ ]:
scatterplot = vistk.Scatterplot(id='sitc4u', color='color', name='sitc4u', x='imports', 
                                y='exports', r='quality_exp', year=1984)
scatterplot.draw(df[df['year'] == 1984])